In [2]:
import sys
print('hello')
sys.path.append('/home/fenics')
sys.path.append('/home/fenics/phd_3/experiments')

# noinspection PyUnresolvedReferences
from dolfin import dx, ds
from dolfin import *
from dolfin.cpp.log import set_log_active
from dolfin.cpp.parameter import parameters
from mshr import Sphere, Box

from default_values import DefaultValues3D
from solver import Problem
from utilities import clear_dir, print_3d_boundaries_on_cube


domain = Box(Point(0., 0.), Point(1., 1.)) - Sphere(Point(0.5, 0.5), .2)
omega = generate_mesh(domain, 100)
plot(omega)

hello


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to Create axis aligned box.
*** Reason:  Box with corner <Point x = 0 y = 0 z = 0> and <Point x = 1 y = 1 z = 0>is degenerate.
*** Where:   This error was encountered inside CSGPrimitives3D.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  74d7efe1e84d65e9433fd96c50f1d278fa3e3f3f
*** -------------------------------------------------------------------------
